In [1]:
import seaborn as sns
dataset=sns.load_dataset('tips')
dataset.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [2]:
dataset['sex'].unique()

['Female', 'Male']
Categories (2, object): ['Male', 'Female']

In [3]:
dataset.shape

(244, 7)

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [5]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()

In [6]:
dataset['time']=encoder.fit_transform(dataset['time'])
dataset['time']

0      0
1      0
2      0
3      0
4      0
      ..
239    0
240    0
241    0
242    0
243    0
Name: time, Length: 244, dtype: int64

In [7]:
dataset.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,0,2
1,10.34,1.66,Male,No,Sun,0,3
2,21.01,3.50,Male,No,Sun,0,3
3,23.68,3.31,Male,No,Sun,0,2
4,24.59,3.61,Female,No,Sun,0,4


In [8]:
dataset['time'].value_counts()

time
0    176
1     68
Name: count, dtype: int64

In [9]:
X=dataset.drop(labels=['time'],axis=1)
X.head()

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4


In [10]:
y=dataset['time']

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=42)

In [12]:
from sklearn.impute import SimpleImputer          # Handling Missing Value
from sklearn.preprocessing import OneHotEncoder   # Handle Categorical value
from sklearn.preprocessing import StandardScaler  # Feature Scaling
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


In [13]:
dataset.columns

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')

In [14]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    int64   
 6   size        244 non-null    int64   
dtypes: category(3), float64(2), int64(2)
memory usage: 8.9 KB


In [15]:
category_column=['sex','smoker','day']
numerical_column=['total_bill','tip','size']

In [16]:
num_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler',StandardScaler())

])
cat_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('onehotencoder',OneHotEncoder())
    ]
)

In [17]:
preprocessor=ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_column),
    ('cat_pipeline',cat_pipeline,category_column)
])

In [18]:
preprocessor

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['total_bill', 'tip', 'size']),
                                ('cat_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder())]),
                                 ['sex', 'smoker', 'day'])])

In [19]:
X_train=preprocessor.fit_transform(X_train)
X_test=preprocessor.transform(X_test)

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [21]:
models={
    'Random_Forest':RandomForestClassifier(),
    'Decission Tree':DecisionTreeClassifier(),
    'SVC':SVC()
}

In [22]:
def evaluate_model(X_train,X_test,y_train,y_test,models):
    report={}
    for i in range(len(models)):
        model=list(models.values())[i]
        model.fit(X_train,y_train)

        y_pred=model.predict(X_test)

        accu_score=accuracy_score(y_test,y_pred)
        accu_score=round(accu_score*100,2)
        report[list(models.keys())[i]]=accu_score
    return accu_score

In [23]:

evaluate_model(X_train,X_test,y_train,y_test,models)

95.92

Now HyperParameter Tunning

In [24]:
classifier=RandomForestClassifier()

params={
    'max_depth':[3,5,8,12],
    'n_estimator':[100,300,600],
    'criterion':['ginni','entropy'],
    'max_features':['sqrt','log2']
}


In [35]:
from sklearn.model_selection import GridSearchCV
hp_model=GridSearchCV(classifier,param_grid=params,scoring='accuracy',cv=5,verbose=3,n_jobs=-1)
hp_model.fit(X_train,y_train)
hp_model.best_params_


Fitting 5 folds for each of 48 candidates, totalling 240 fits


ValueError: Invalid parameter 'n_estimator' for estimator RandomForestClassifier(criterion='ginni', max_depth=3). Valid parameters are: ['bootstrap', 'ccp_alpha', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'max_samples', 'min_impurity_decrease', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'monotonic_cst', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'].

In [33]:
classifier=RandomForestClassifier(criterion='gini',max_depth=3,max_features='sqrt',n_estimators=100)
classifier.fit(X_train,y_train)
y_pred=classifier.predict(X_test)
accuracy_score(y_test,y_pred)


1.0

In [34]:
classifier.score(X_train,y_train)

0.958974358974359